In [ ]:
%env JAVA_TOOL_OPTIONS='--add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.lang=ALL-UNNAMED'
%env FLINK_CONF_DIR=/Users/lgfquentin/dev/medical-dashboard/back-end-synthea-flink/config

In [ ]:
%load_ext streaming_jupyter_integrations.magics

In [ ]:
%flink_connect --execution-target remote --remote-hostname localhost --remote-port 8081
#%flink_connect 

In [ ]:
%%flink_execute_sql
CREATE OR REPLACE TABLE hospital (
  `id` STRING,
  `identifier` ROW (`value` STRING) ARRAY,
  `status` STRING,
  `name` STRING,
  `telecom` ROW (`system` STRING, `value` STRING) ARRAY,
  `address` ROW (
    `line` STRING ARRAY,
    `city` STRING,
    `state` STRING,
    `postalCode` STRING,
    `country` STRING
  ),
  `position` ROW (
    `longitude` DOUBLE,
    `latitude` DOUBLE
  ),
  `managingOrganization` ROW (
    `identifier` ROW (`value` STRING)
  )
) WITH (
  'connector' = 'kafka',
  'topic' = 'Location',
  'properties.bootstrap.servers' = 'kafka-edge1:9092',
  'properties.group.id' = 'testGroup',
  'scan.startup.mode' = 'earliest-offset',
  'format' = 'json'
)

In [ ]:
%%flink_execute_sql
CREATE TABLE hospital_psql (
  `id_organization` STRING,
  `id_location` STRING,
  `status` STRING,
  `name` STRING,
  `telecom` STRING,
  `address_line` STRING,
  `city` STRING,
  `state` STRING,
  `postalCode` STRING,
  `country` STRING,
  `longitude` DOUBLE,
  `latitude` DOUBLE,
  PRIMARY KEY (`id_organization`) NOT ENFORCED
) WITH (
  'connector' = 'jdbc',
  'url' = 'jdbc:postgresql://db:5432/example',
  'table-name' = 'public.hospital',
  'username' = 'example',
  'password' = 'example'
);


In [ ]:
%%flink_execute_sql
INSERT INTO hospital_psql
SELECT 
  h.managingOrganization.identifier[1] as id_organization,
  h.id as id_location, 
  status, 
  name, 
  h.telecom[1][1] AS telecom, 
  h.address.line[1] AS address,  
  h.address.city AS city, 
  h.address.state AS state, 
  h.address.country AS country, 
  h.address.postalCode AS postalCode, 
  h.`position`.longitude AS longitude,
  h.`position`.latitude AS latitude
  FROM 
  hospital h WHERE h.managingOrganization.identifier[1] <>'None';